In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle
import gzip
import os
import glob
import datetime
import warnings
warnings.filterwarnings('ignore')

np.random.seed(7)
from colorama import Fore
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error

In [3]:
weather_columns = ['date', 'avg_wind_speed', 'peak_gust_time', 'precipitation', 'snow', 'snow_depth', 'temp_avg', 'temp_max', 'temp_min', 'tot_sunshine',
                  'dir_fwind_2min', 'dir_fwind_5min', 'speed_fwind_2min', 'speed_fwind_5min', 'fog', 'heavy_fog', 'thunder', 'ice_pellets', 'hail', 'glaze', 'smoke']
traffic_columns = ['PUZone', 'Count', 'PUTime']

In [4]:
from datetime import datetime, timedelta

def datetime_range(end, delta, count):
        
        current = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
        for i in range(count):
            yield current
            current -= delta

In [5]:
from sklearn.model_selection import cross_validate
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        "Cross validation Mean Absolute Error:",mae
    )


In [6]:
high_zone = [0] * 73
for h in [1, 15, 24, 25, 26, 30, 31, 37, 41, 43, 44, 46, 62, 69]:
    high_zone[h] = 1

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
class Model():
    
    def __init__(self):
        self.random_forest = RandomForestRegressor(n_estimators=100)
        self.decision_tree = DecisionTreeRegressor()
        self.mlp = MLPRegressor(hidden_layer_sizes=(300,150), max_iter=200,activation ='relu',solver='adam',random_state=1)
        self.xgb = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
        self.lgbm = LGBMRegressor(learning_rate = 0.01, num_iterations = 1000)
        self.svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
#         self.ensemble = VotingRegressor([('lgbm', self.lgbm), ('xgb', self.xgb), ('svr', self.svr)])
        self.ensemble = VotingRegressor([('lgbm', self.lgbm), ('xgb', self.xgb), ('rf', self.random_forest), ('mlp', self.mlp)])
        self.count = 0
        self.to_keep = []
        self.weather_to_keep = []
        self.train_drop_list = ['Count(0)', 'date']
        # self.model = self.ensemble
        self.model = self.mlp
        return 
    
    def train_model(self, traffic, weather):
        df = self.preprocessing(traffic, weather)
        df.to_csv('data/new_merged.csv', index=False)
        
        train = df.loc[df.date < '2017-05-01']
        valid = df.loc[df.date >= '2017-05-01']
        
        ts_cv = TimeSeriesSplit(
            n_splits=5,
            gap=48,
            max_train_size=10000,
            test_size=1000,
        )
        
        train_y = train['Count(0)']
        
        train_x = train.drop(self.train_drop_list, axis=1)
       
        self.to_keep = train_x.columns
        
        valid_y = valid['Count(0)']
        
        valid_x = valid.drop(self.train_drop_list, axis=1)
       
        self.model.fit(train_x,train_y)
        pred_y = self.model.predict(valid_x)
        valid_y_array = valid_y.values.ravel()
        mae = mean_absolute_error(valid_y_array,pred_y)
        print('Validation MAE: ', mae)
        evaluate(self.model, df.drop(self.train_drop_list, axis=1), df['Count(0)'], cv=ts_cv)
    
    def preprocessing(self, traffic, weather):
        
        
        # extract date and hour
        
        traffic['PUTime'] = pd.to_datetime(traffic['PUTime'])
        traffic['date'] = traffic['PUTime'].dt.date
        traffic['hour'] = traffic['PUTime'].dt.hour
        traffic['weekday'] = (traffic['PUTime'].dt.dayofweek < 5).astype(int)
        
        traffic['peak_hour'] = (traffic['hour'] >= 16) * (traffic['hour'] <= 20) + (traffic['weekday'] == 1) * (traffic['hour'] >= 6) * (traffic['hour'] <= 10) 
        traffic['peak_hour'] = traffic['peak_hour'].astype(int)
        
        # group by zones
        grouped = traffic.groupby(traffic.PUZone)
        dfs = []
        for i in range(0,73):
            dfs.append(grouped.get_group(i))
        
        for zone in range(0,73) :
            dfs[zone].drop(['PUZone'], axis=1,inplace=True)
            
        
        
        # Preprocess weather
        weather.columns = weather_columns

        weather['date'] = pd.DatetimeIndex(weather['date']).date
        
        # handle null values
        weather = weather.dropna(how='all')
        weather = weather.drop(['peak_gust_time', 'temp_avg', 'tot_sunshine', 'thunder', 'ice_pellets', 'hail', 'glaze'], 1)
        
        ### replace with 0
        for col in weather.columns:
            weather[col] = weather[col].fillna(0)
        
                                
        weather = weather[['date']+self.weather_to_keep]
        
        X2 = []
        # add same hour for previous 30 days
        for zone in range(0,73) :
            dfs[zone]["PUTime"] = pd.to_datetime(dfs[zone]["PUTime"]) # Convert column type to be datetime
            indexed_df = dfs[zone].set_index(["PUTime"])           # Create a datetime index
            indexed_df.drop(['date'],axis=1,inplace=True)
            indexed_df.drop(['hour'],axis=1,inplace=True)
            indexed_df.drop(['weekday'], axis=1, inplace=True)
            indexed_df.drop(['peak_hour'], axis=1, inplace=True)
            
            
            n_steps_in = 30
            X_new = pd.DataFrame()
            for k in range(len(indexed_df.columns)) :
                for i in range(0, n_steps_in):
                    for j in range(-1, 1, 1):
                        X_new[indexed_df.columns[k] + '(' + str(-24*i+j) + ')'] = indexed_df.iloc[:,k].shift(24*i-j)
                    if i<=24:
                        X_new[indexed_df.columns[k] + '(' + str(-i) + ')'] = indexed_df.iloc[:,k].shift(i)
                X_new[indexed_df.columns[k] + '(' + str(-24*30) + ')'] = indexed_df.iloc[:,k].shift(24*30)
            X_new = X_new.iloc[:, ::-1]
            X_new['date'] = dfs[zone]['date'].values
            X_new['hour'] = dfs[zone]['hour'].values
            X_new['weekday'] = dfs[zone]['weekday'].values
            X_new['peak_hour'] = dfs[zone]['peak_hour'].values
            
            X2.append(X_new)
            
            
        for zone in range(0,73):
            X2[zone] = X2[zone].dropna()
            
        dfW2 = []
        for zone in range(0,73):
            X2[zone]['date'] = X2[zone]['date'].astype(str)
            weather['date'] = weather['date'].astype(str)
            dfW2.append(X2[zone].merge(weather, on='date'))
        
        
        for zone in range(0,73):
            dfW2[zone]['PUZone'] = zone
            dfW2[zone]['highZone'] = high_zone[zone]
            
        
        dfAll2 = pd.concat(dfW2, axis=0)
        return dfAll2

    def preprocess_test(self, demand, weather, dt, neighbour):
        
        df = pd.DataFrame(columns=self.to_keep)
        df['PUZone'] = [i for i in range(0, 73)]
        df['highZone'] = high_zone
        
        pred_date_time_obj = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        pred_date, pred_time = dt.split(" ")
        df['weekday'] = (pred_date_time_obj.weekday() < 5)
        df['weekday'] = df['weekday'].astype(int)
        df['hour'] = pred_date_time_obj.hour
        
        df['peak_hour'] = (df['hour'] >= 16) * (df['hour'] <= 20) + (df['weekday'] == 1) * (df['hour'] >= 6) * (df['hour'] <= 10)
        df['peak_hour'] = df['peak_hour'].astype(int)
        
        
        for i in range(1, 25):
            df['Count('+str(-i)+')'] = demand[-i]
        for i in range(1, 30):
            for j in range(-1, 1, 1):
                df['Count('+str(-i*24+j)+')'] = demand[-i*24+j]
        df['Count('+str(-30*24)+')'] = demand[-30*24]
        
        w = weather[-1]
        for i in range(len(weather_columns)):
            if weather_columns[i] in self.to_keep:
                df[weather_columns[i]] = [w[i]] * 73
                df[weather_columns[i]] = df[weather_columns[i]].interpolate()
        
        
        # fill nulls
        df = df.fillna(0)
        return df
    

    def predict(self, demand, weather, dt, neighbors):
        '''
        Parameters
        ----------
        demand: (24*30, 73) numpy array containing last 30 days' hourly demand data, e.g. demand[-1, 3] contains last hour's demand of zone 3
        weather: List of lists containing today's and last 30 days' weather data, e.g., weather[-1] is a list containing today's weather data with [DATE, AWND,...,WT08] as in weather.csv
        dt: date and time of the prediction e.g., "2017-06-01 00:00:00"
        neighbors - Dictionary containing the mapping between each zone and their list of neighbors in zone_neighbors.json

        Return
        ------
        predictions: List of 73 non-negative integers - your trip forecast for each zone in the next hour
        '''
        df = self.preprocess_test(demand, weather, dt, neighbors)
        pred_y = self.model.predict(df)
        
        self.count += 1
        if self.count % 100 == 0:
            print(self.count, ' times done.')
        
        return pred_y

In [9]:
traffic = pd.concat(map(pd.read_csv, ['data/2017-01_1H_zone.csv', 'data/2017-02_1H_zone.csv', 
                                     'data/2017-03_1H_zone.csv', 'data/2017-04_1H_zone.csv', 'data/2017-05_1H_zone.csv']))
traffic.drop(['Unnamed: 0'], axis=1,inplace=True)
        
weather = pd.read_csv('data/weather.csv')

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
model = Model()
model.train_model(traffic, weather)

Validation MAE:  15.319702106394798
Cross validation Mean Absolute Error: [ 7.84554416  7.66867243 24.17955808 25.13287821 26.99096867]


In [51]:

def test_pred_eval(model, test, test_y):
    test_size = len(test)

    test_preds = []
    for i in range(len(test)):
        test_preds.append(model.predict(test[i]['demand'], test[i]['weather'], test[i]['dt'], test[i]['neighbors']))

    test_preds = np.array(test_preds)
    test_y = np.array(test_y)    

    mae = mean_absolute_error(test_y.flatten(), test_preds.flatten())

    zone_mae = []
    for i in range(test_y.shape[1]):
        zone_mae.append(mean_absolute_error(test_y[:, i], test_preds[:, i]))
    return mae, zone_mae
    


In [ ]:
with gzip.open("data/test.pkl.gz") as file:
    test = pickle.load(file)

with gzip.open("data/test_answer.pkl.gz") as file:
    test_y = pickle.load(file)

In [52]:
mae, zone_mae = test_pred_eval(model, test, test_y)

3700  times done.
3800  times done.
3900  times done.
4000  times done.
4100  times done.
4200  times done.
4300  times done.


In [53]:
print(mae)

14.882286495321218


In [54]:
print(zone_mae)

[23.471581879538366, 24.01970769157133, 17.05162749608661, 17.121360319977647, 4.2603809348181105, 0.5333122900184656, 2.5013185948039025, 0.1705492886768105, 10.186196491202834, 0.958298215826876, 12.551814352885998, 0.19017128883962897, 15.926841566672, 16.837179113567394, 22.24757975487184, 33.379830193561254, 12.939551378918534, 14.70561346246038, 3.463089773570776, 17.49538328247685, 11.581026821598655, 3.764335321702044, 16.891869174890154, 0.6243375923464205, 43.48820442077819, 36.17483886949751, 28.772775920388106, 24.00111886672809, 10.13500492176678, 1.3502378320256643, 30.61728977855632, 44.91674299830498, 0.4842245571704069, 4.939997326735158, 11.869013797794777, 6.313619024956076, 18.982415124877637, 33.92051275038255, 20.09918594614908, 4.711172216167884, 17.64829407859552, 29.84384516523149, 0.30217681426151144, 34.28653457584131, 37.795697353211125, 22.80233736072584, 26.1153193483182, 6.52643118355413, 1.427991687687768, 3.7952372262992373, 21.90837358636295, 1.3803322